In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DateType, TimestampType

In [0]:
spark = SparkSession.builder.appName('ops').getOrCreate()

In [0]:
path = '/FileStore/tables/appl_stock.csv'
df = spark.read.csv(path, inferSchema = True, header = True)

In [0]:
df.printSchema()

root
-- Date: string (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: double (nullable = true)

In [0]:
df.withColumn('Date', df.Date.cast(TimestampType()))

Out[10]: DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: int, Adj Close: double]

In [0]:
df.head(3)[0]

Out[11]: Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

In [0]:
df.filter(df['Close']<500).select('Volume').show() # one condition

+---------+
 Volume|
+---------+
123432400|
150476200|
138040000|
119282800|
111902700|
115557400|
148614900|
151473000|
108223500|
148516900|
182501900|
153038200|
152038600|
220441900|
266424900|
466777500|
430642100|
293375600|
311488100|
187469100|
+---------+
only showing top 20 rows

In [0]:
df.filter((df['Close']<200) & (df['Open']>200)).show() # add parentheses for every condition

+----------+------------------+----------+----------+----------+---------+------------------+
 Date| Open| High| Low| Close| Volume| Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
2010-01-22|206.78000600000001|207.499996| 197.16| 197.75|220441900| 25.620401|
2010-01-28| 204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
2010-01-29| 201.079996|202.199995|190.250002|192.060003|311488100| 24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+

In [0]:
result = df.filter(df['Low'] == 197.16).collect()

In [0]:
row = result[0]

In [0]:
row.asDict()

Out[19]: {'Date': '2010-01-22',
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [0]:
data = spark.read.csv('/FileStore/tables/sales_info.csv', inferSchema = True, header = True)

In [0]:
data.show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG| Sam|200.0|
 GOOG|Charlie|120.0|
 GOOG| Frank|340.0|
 MSFT| Tina|600.0|
 MSFT| Amy|124.0|
 MSFT|Vanessa|243.0|
 FB| Carl|870.0|
 FB| Sarah|350.0|
 APPL| John|250.0|
 APPL| Linda|130.0|
 APPL| Mike|750.0|
 APPL| Chris|350.0|
+-------+-------+-----+

In [0]:
data.printSchema()

root
-- Company: string (nullable = true)
-- Person: string (nullable = true)
-- Sales: double (nullable = true)

In [0]:
data.groupBy('Company').mean().show()

+-------+-----------------+
Company| avg(Sales)|
+-------+-----------------+
 APPL| 370.0|
 GOOG| 220.0|
 FB| 610.0|
 MSFT|322.3333333333333|
+-------+-----------------+

In [0]:
data.agg({'Sales':'sum'}).show()

+----------+
sum(Sales)|
+----------+
 4327.0|
+----------+

In [0]:
group_data = data.groupBy('Company')

In [0]:
group_data.agg({'Sales':'max'}).show()

+-------+----------+
Company|max(Sales)|
+-------+----------+
 APPL| 750.0|
 GOOG| 340.0|
 FB| 870.0|
 MSFT| 600.0|
+-------+----------+

In [0]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [0]:
data.select(countDistinct('Sales').alias('distinct count of sales')).show()

+-----------------------+
distinct count of sales|
+-----------------------+
 11|
+-----------------------+

In [0]:
data.select(stddev('Sales')).show()

+------------------+
stddev_samp(Sales)|
+------------------+
250.08742410799007|
+------------------+

In [0]:
from pyspark.sql.functions import format_number

In [0]:
sales_std = data.select(stddev('Sales').alias('std')) #format column label

In [0]:
sales_std.show()

+------------------+
 std|
+------------------+
250.08742410799007|
+------------------+

In [0]:
sales_std.select(format_number('std', 2).alias('std')).show() # format column number decimal places

+------+
 std|
+------+
250.09|
+------+

In [0]:
data.orderBy('Sales').show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 GOOG|Charlie|120.0|
 MSFT| Amy|124.0|
 APPL| Linda|130.0|
 GOOG| Sam|200.0|
 MSFT|Vanessa|243.0|
 APPL| John|250.0|
 GOOG| Frank|340.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 MSFT| Tina|600.0|
 APPL| Mike|750.0|
 FB| Carl|870.0|
+-------+-------+-----+

In [0]:
data.orderBy(data['Sales'].desc()).show()

+-------+-------+-----+
Company| Person|Sales|
+-------+-------+-----+
 FB| Carl|870.0|
 APPL| Mike|750.0|
 MSFT| Tina|600.0|
 FB| Sarah|350.0|
 APPL| Chris|350.0|
 GOOG| Frank|340.0|
 APPL| John|250.0|
 MSFT|Vanessa|243.0|
 GOOG| Sam|200.0|
 APPL| Linda|130.0|
 MSFT| Amy|124.0|
 GOOG|Charlie|120.0|
+-------+-------+-----+